In [ ]:
'''
【项目03】  知乎数据清洗整理和结论研究

作业要求：
1、数据清洗 - 去除空值
要求：创建函数
提示：fillna方法填充缺失数据，注意inplace参数

2、问题1 知友全国地域分布情况，分析出TOP20
要求：
① 按照地域统计 知友数量、知友密度（知友数量/城市常住人口），不要求创建函数
② 知友数量，知友密度，标准化处理，取值0-100，要求创建函数
③ 通过多系列柱状图，做图表可视化
提示：
① 标准化计算方法 = (X - Xmin) / (Xmax - Xmin)
② 可自行设置图表风格

3、问题2 知友全国地域分布情况，分析出TOP20
要求：
① 按照学校（教育经历字段） 统计粉丝数（‘关注者’）、关注人数（‘关注’），并筛选出粉丝数TOP20的学校，不要求创建函数
② 通过散点图 → 横坐标为关注人数，纵坐标为粉丝数，做图表可视化
③ 散点图中，标记出平均关注人数（x参考线），平均粉丝数（y参考线）
提示：
① 可自行设置图表风格

'''

import os 
import pandas as pd
import numpy as np
os.chdir("C:/Users/CT/PycharmProjects/jupyter/data_analysor/project3/")
os.listdir()

#导入数据
#此处必须指定engine为python，否则默认为C engine，会导致错误
sixpu_dat=pd.read_csv("./六普常住人口数.csv",engine='python')
zhihu_dat=pd.read_csv("./知乎数据_201701.csv",engine='python')

sixpu_dat

zhihu_dat.head()

print(zhihu_dat.loc[0,'个人简介'],type(zhihu_dat.loc[0,'个人简介']))

'''
1、数据清洗 - 去除空值
要求：创建函数
提示：fillna方法填充缺失数据，注意inplace参数
'''
#此处需要针对不同类型的列采用不同的空值填充策略
def removeNaN(df):
    for i in df.columns:
        if(df[i].dtype=='object'):
            df[i].fillna('缺失数据',axis=0,inplace=True)
        else:
            df[i].fillna(0,axis=0,inplace=True)
    return df

new_zhihu_dat=removeNaN(zhihu_dat)
new_zhihu_dat.head(10)

'''
2、问题1 知友全国地域分布情况，分析出TOP20
要求：
① 按照地域统计 知友数量、知友密度（知友数量/城市常住人口），不要求创建函数
② 知友数量，知友密度，标准化处理，取值0-100，要求创建函数
③ 通过多系列柱状图，做图表可视化
提示：
① 标准化计算方法 = (X - Xmin) / (Xmax - Xmin)
② 可自行设置图表风格
'''
new_zhihu_dat['居住地'].unique()

#按照地域统计 知友数量/知友密度
df_city=new_zhihu_dat.groupby('居住地').count()
print(df_city.head(10))
sixpu_dat['city']=sixpu_dat['地区'].str[:-1]
sixpu_dat.head()

merge_dat=pd.merge(df_city,sixpu_dat,left_index=True,right_on='city')[['_id','city','常住人口']]
merge_dat

#求知友数量和知友密度
merge_dat['知友密度']=merge_dat['_id']/merge_dat['常住人口']
merge_dat.sort_values(by='_id',ascending=False)

#归一化处理

def scaleProcess(df,*cols):
    colnames=[]
    for i in cols:
        df[i+'_nor']=(df[i]-min(df[i]))/(max(df[i])-min(df[i]))*100
        colnames.append(i+'_nor')
    return (df,colnames)

result=scaleProcess(merge_dat,'_id','知友密度')
result_df=result[0]
#按知友数量top前20
top20_id_dat=result[0].sort_values(result[1][0],ascending=False)
top20_dense_dat=result[0].sort_values(result[1][1],ascending=False)

df1=top20_id_dat[['city','_id_nor']][:20]
df2=top20_dense_dat[['city','知友密度_nor']][:20]
print(top20_id_dat[['city','_id_nor']][:5])
print(top20_dense_dat[['city','知友密度_nor']][:5])

#构建多系列柱状图
import matplotlib.pyplot as plt
#解决标签乱码问题
plt.rcParams['font.sans-serif']=['SimHei']
plt.rcParams['axes.unicode_minus']=False

fig1=plt.figure(num=1,figsize=(10,5))
plt.bar(range(20),df1['_id_nor'],color='red',tick_label=df1['city'],alpha=0.6)
for i,j in zip(range(20),df1['_id_nor']):
    plt.text(i-0.3,1,'%.0f'%j,color='blue')
plt.title("知友数量top20")

fig2=plt.figure(num=2,figsize=(10,5))
plt.bar(range(20),df2['知友密度_nor'],color='blue',tick_label=df2['city'],alpha=0.6)
for i,j in zip(range(20),df2['知友密度_nor']):
    plt.text(i-0.3,1,'%.0f'%j,color='black')
plt.title("知友密度top20")

'''
3、问题2 知友全国地域分布情况，分析出TOP20
要求：
① 按照学校（教育经历字段） 统计粉丝数（‘关注者’）、关注人数（‘关注’），并筛选出粉丝数TOP20的学校，不要求创建函数
② 通过散点图 → 横坐标为关注人数，纵坐标为粉丝数，做图表可视化
③ 散点图中，标记出平均关注人数（x参考线），平均粉丝数（y参考线）
提示：
① 可自行设置图表风格

'''

#按照学校统计粉丝数
df_area=zhihu_dat[['教育经历','关注者','关注']]
df_area.columns=['学校','粉丝数','关注人数']
df_area.head(10)

df_area.groupby('学校').sum()[['粉丝数','关注人数']].sort_values(by='粉丝数',ascending=False)[:25]
q_data=df_area.groupby('学校').sum()[['粉丝数','关注人数']].drop(['缺失数据','为往圣继绝学','医学','我的老師，是山川和大地','本科']).sort_values(by='粉丝数',ascending=False)[:20]
q_data

x,y=q_data['粉丝数'],q_data['关注人数']
plt.figure(figsize=(12,5))
plt.scatter(x,y,marker='.',s=y/100,cmap='Reds',c=x,label='学校',)

cared,care=q_data['粉丝数'].mean(),q_data['关注人数'].mean(),
plt.axhline(care,label='平均关注人数%d人'%care,color='red')
plt.axvline(cared,label='平均粉丝数%d人'%cared,color='blue')
plt.legend()

for i,j,n in zip(x,y,q_data.index):
    plt.text(i+500,j,n)

